# AESTHEMOS
For use with data collected from 

## TO-DO:
### Fix if subject not present in file
### add IDs of paintings
### Re-order columns before saving CSV
### do actual computation work lol

In [2]:
import pandas as pd
from numpy import array
import os

In [3]:
# Fixes numerical values in gallery files AND makes unique subject list
unique_subjects = set()
for gallery in os.listdir('data/aesthemos'):
    if gallery.endswith(".csv"):
        g = pd.read_csv("data/aesthemos/" + gallery).replace({'1 - Not at all':1, '7 - Strongly':7})
        g_subs = g['Subject #:'].tolist()
        z = [x.replace(' ', '') for x in g_subs]
        unique_subjects = unique_subjects.union(set(z))
        g.to_csv("data/aesthemos/" + gallery, index=False)
list(unique_subjects)

['pilot02', 'pilot01']

In [11]:
for gallery in os.listdir('data/aesthemos'):
    if gallery.endswith(".csv"):
        print(gallery.split()[1].replace(".csv", ""))

019
003
002
016
200
001
100


In [302]:
gal1 = raw_data = pd.read_csv("data/aesthemos/Gallery 001.csv")
gal2 = raw_data = pd.read_csv("data/aesthemos/Gallery 002.csv")
gal3 = raw_data = pd.read_csv("data/aesthemos/Gallery 003.csv")

In [288]:
raw_data = pd.read_csv("data/aesthemos/Gallery 003.csv")
raw_data

,Timestamp,Subject #:,Woman Seated by Hans Hofmann [Moved],Woman Seated by Hans Hofmann [Fascinated],Woman Seated by Hans Hofmann [Funny],Woman Seated by Hans Hofmann [Surprised],Woman Seated by Hans Hofmann [Indifferent],Woman Seated by Hans Hofmann [Calm],Woman Seated by Hans Hofmann [Unsettled],Woman Seated by Hans Hofmann [Beautiful],...,Still-life with Peaches by Raphaelle Peale [Unsettled],Still-life with Peaches by Raphaelle Peale [Beautiful],Still-life by José Agustin Arrieta [Moved],Still-life by José Agustin Arrieta [Fascinated],Still-life by José Agustin Arrieta [Funny],Still-life by José Agustin Arrieta [Surprised],Still-life by José Agustin Arrieta [Indifferent],Still-life by José Agustin Arrieta [Calm],Still-life by José Agustin Arrieta [Unsettled],Still-life by José Agustin Arrieta [Beautiful]
0,2020/04/07 2:00:48 PM MDT,pilot01,4,5,5,5,3,3,1,4,...,1,5,3,3,2,2,5,4,7,3
1,2020/04/08 1:27:20 PM MDT,pilot02,1,1,4,2,3,4,2,2,...,1,5,3,4,1,4,2,5,1,4


In [12]:
def clean_single_gallery(file, sub):
    sub_ser = pd.Series([x.replace(' ', '') for x in file['Subject #:']])
    
    table = file.loc[sub_ser == sub].T.reset_index().drop([0,1]).rename(columns={"index": "Title"})
    # if rows = 0 return empty df UGNSODFN:EJNOT EP"OJGSE{JP<ES} IF STATEMENT NOT WORKING
    if len(table.columns) > 1:
        table = (
            table
            .assign(category=table.Title.apply(lambda x: x.split(" ")[-1]))
            .assign(Name=table.Title.apply(lambda x: " ".join(x.split(" ")[:-1])))
            .rename(columns={ table.columns[-1]: "score"})
            .drop('Title', axis=1) 
         )
        table = table.assign(score=table.score.astype(int))
        t = table.pivot_table(index='Name', columns='category', aggfunc=(lambda x: x))
        t.columns = t.columns.levels[1].rename(None)
        t = t.rename(columns=lambda x: x[1:-1]).reset_index()
        art_split = t.Name.str.split(" by ", expand=True)
        t['Title'] = art_split[0]
        t['Artist'] = art_split[1]
        t = t.drop('Name', axis=1)
        t['Gallery'] = file.split()[1].replace(".csv", "")
        # GET IDs FROM TEMPLATE (Maybe make a csv 'key' file?)
        return t
    else:
        print(sub + " not in " + file)

In [13]:
for subject in unique_subjects:
    current_sub = pd.DataFrame(
        columns = ["Title", "Artist" "Beautiful", "Calm", "Fascinated", "Funny", "Indifferent", "Moved", "Surprised", "Unsettled"]
    )
    for gallery in os.listdir('data/aesthemos'):
        if gallery.endswith(".csv"):
            g_csv = pd.read_csv("data/aesthemos/" + gallery)
            fix_curr = clean_single_gallery(g_csv, subject)
            current_sub = current_sub.append(fix_curr, ignore_index = True)
    # RE-ORDER COLUMNS
    current_sub.to_csv("output/"+ subject + "-groups.csv", index=False)
        
fix_curr

AttributeError: 'DataFrame' object has no attribute 'split'

In [ ]:
# for loop processing each file to add computation stuff needed for catagories